In [7]:
!pip install pandas
!pip install plotly
!pip install chart-studio
!pip install statsmodels

In [6]:
import pandas as pd
import datetime as dt
import numpy as np
import math
import glob
import datetime
from pandas import datetime
from matplotlib import pyplot
import itertools
from scipy import integrate
import warnings

# if using a Jupyter notebook, inlcude:
%matplotlib inline

In [3]:
# Import used libraries
# Initialitation of variables
path ='./data/unprocessed/CSO_*__*_*_*.csv' # Data path schema
allFiles = glob.glob(path) # Get all files
IndexSlice = pd.IndexSlice

# Columns names per file type
columnNamesPBC=['Timestamp', 'Pump', 'State', 'Starts count', 'Total Hours', 'Last Maintenance Hours', 'Last reset Hours']

# Columns types file type
typesDictPBC = {'Pump': 'O', 'State': 'float64', 'Starts count': 'float64', 'Total Hours': 'float64', 'Last Maintenance Hours': 'float64', 'Last reset Hours': 'float64'}

# Buffer variables
bufferPBC = []

# List of stations and pumps
stationList = []
pumpList = ['04501', '04502', '04503', '04504', '04505', '04601', '04602', '04603', '04604', '04701', '04702', '04703', '04801', '04802']

# Auxiliary functions for data transform
pumpStateMapBeautify = {
    1: 'run', 
    2: 'stop', 
    3: 'run', 
    4: 'stop', 
    5: 'run', 
    6: 'stop', 
    7: 'unavailable', 
    99: 'failure', 
    -1: 'maintenance'
}
pumpStartValue = {
    'run': 1.0,
    'stop': 0.0,
    'unavailable': 0.0,
    'failure': 0.0,
    'maintenance': 0.0
}
pumpStart = {
    -1: 'start',
    1: 'stop',
    0: 'no change'
}
                

# Procesa todos los ficheros
for filename in allFiles:
    # Split the file name to extract the pump and station
    nameParts = filename.split('_')
    station = nameParts[1]
    pump = nameParts[4]

    # Add pump and stations to list
    if station not in stationList:
        stationList.append(station)
    
    # PBC files: files with station flow
    if nameParts[3]=='PBC':
        data = pd.read_csv(filename, sep=';', parse_dates=['Timestamp'], names=columnNamesPBC, header=0, na_values=['', ' ', 0], dtype=typesDictPBC);
        data['Station'] = station
        data['Pump'] = pump
        data['STS'] = data['State']
        data['State'] = data['State'].map(pumpStateMapBeautify)
        data['State value'] = data['State'].map(pumpStartValue)
        data['Start'] = 0
        bufferPBC.append(data)

In [4]:
bufferPBC

[                Timestamp   Pump        State  Starts count  Total Hours  \
 0     2016-01-01 00:14:51  04501         stop           NaN          NaN   
 1     2016-01-01 02:09:21  04501      failure           NaN          NaN   
 2     2016-01-01 02:10:51  04501         stop           NaN          NaN   
 3     2016-01-01 02:19:41  04501          run           NaN          NaN   
 4     2016-01-01 02:20:32  04501          NaN        4563.0          NaN   
 ...                   ...    ...          ...           ...          ...   
 11593 2016-01-31 18:05:22  04501          NaN           NaN          NaN   
 11594 2016-01-31 18:06:42  04501  unavailable           NaN          NaN   
 11595 2016-01-31 18:26:42  04501         stop           NaN          NaN   
 11596 2016-01-31 20:37:22  04501  unavailable           NaN          NaN   
 11597 2016-01-31 21:54:42  04501         stop           NaN          NaN   
 
        Last Maintenance Hours  Last reset Hours Station   STS  State valu

In [5]:
# Concatenate and transform PBC buffer
PBCDataSet = pd.concat(bufferPBC, axis=0, ignore_index='True')
PBCDataSet = PBCDataSet.sort_values(by=['Timestamp'])
bufferPBCPump = []
for pump in pumpList:
    data = PBCDataSet.loc[PBCDataSet['Pump']==pump]
    data = data.fillna(method='ffill')
    data = data.fillna(method='backfill')
    data['Start'] = data['State value'].diff()
    data.fillna(0, inplace=True)
    bufferPBCPump.append(data)
PBCDataSet = pd.concat(bufferPBCPump, axis=0, ignore_index='True')
PBCDataSet['DayOfWeek'] = PBCDataSet['Timestamp'].dt.dayofweek
PBCDataSet['WeekOfYear'] = PBCDataSet['Timestamp'].dt.weekofyear
PBCDataSet['Month'] = PBCDataSet['Timestamp'].dt.month
PBCDataSet = PBCDataSet.set_index(['Timestamp','Station', 'Pump'])
PBCDataSet.head()

,,,State,Starts count,Total Hours,Last Maintenance Hours,Last reset Hours,STS,State value,Start,DayOfWeek,WeekOfYear,Month
Timestamp,Station,Pump,,,,,,,,,,,
2016-01-01 00:14:51,45CLMN,04501,stop,4563.0,0.11,1153.860107,6716.0,2.0,0.0,0.0,4,53,1
2016-01-01 02:09:21,45CLMN,04501,failure,4563.0,0.11,1153.860107,6716.0,99.0,0.0,0.0,4,53,1
2016-01-01 02:10:51,45CLMN,04501,stop,4563.0,0.11,1153.860107,6716.0,2.0,0.0,0.0,4,53,1
2016-01-01 02:19:41,45CLMN,04501,run,4563.0,0.11,1153.860107,6716.0,1.0,1.0,1.0,4,53,1
2016-01-01 02:20:32,45CLMN,04501,run,4563.0,0.11,1153.860107,6716.0,1.0,1.0,0.0,4,53,1
